In [4]:
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter.messagebox import showinfo
from tkinter import filedialog as fd
from PIL import ImageTk,Image  
import PIL

from collections import Counter
import numpy as np
from pylab import *
import matplotlib.pyplot as plt
import math
import struct
import os

In [5]:
### Reading and Writing PGM Files (Greyscale Images) ###

def readpgm2(name):
# Reads a pgm file (ASCII/P2) 
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    with open(name) as f:
        lines = f.readlines()
    for l in list(lines):
        if l[0] == '#':
            lines.remove(l)
    assert lines[0].strip() == 'P2' 
    arr = []
    for li in lines[1:]:
        s_li = li.split()
        i_li = list(map(lambda x: int(x), s_li))
        arr.append(i_li)
    return arr

def readpgm5(name):
# Reads a pgm file (binary/P5)
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    f = open(name, "rb")
    header = []
    w=0
    h=0
    while(len(header)<3):
        temp = f.readline()
        if temp[0]==35:
            print("Comment detected in " + name)
        elif len(header)==0:
            assert (temp == b'P5\n')
            header.append(temp.decode("utf-8"))
        elif len(header)==1:
            (w, h) = [int(i.decode("utf-8")) for i in temp.split()]
            header.append([w,h])
        elif len(header)==2:
            header.append([int(temp)])

    data = []
    data.append(header[1])
    data.append(header[2])
    for y in range(h):
        row = []
        for y in range(w):
            row.append(ord(f.read(1)))
        data.append(row)
    return data

def readpgm(name):
# Reads a pgm file (binary or ascii)
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]
    try:
        try:
            return readpgm5(name)
        except:
            return readpgm2(name)
    except:
        print("Error reading " + name)
        return

def writepgm(file_name, data):
# Writes a pgm file (binary/P5) given pgm data with header

    file_handle = open (file_name, 'wb')
    w = data[0][0]
    h = data[0][1]
    gray = data [1][0]
    pgm_header = f'P5\n{w} {h}\n{gray}\n'
    
    file_handle.write (bytearray (pgm_header, 'ascii')) 

    grayV = np.reshape (data[2:], w*h)

    grayB = struct.pack ('%sB' % len(grayV), *grayV)
    file_handle.write(grayB)
    file_handle.close()
    return


In [36]:
### Reading and Writing PPM Files (Full-Color Images) ###

def readppm3(name):
# Reads a ppm file (ASCII/P3) 
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    with open(name) as f:
        lines = f.readlines()
    for l in list(lines):
        if l[0] == '#':
            lines.remove(l)
    assert lines[0].strip() == 'P3' 
    arr = []
    for li in lines[1:]:
        s_li = li.split()
        i_li = list(map(lambda x: int(x), s_li))
        arr.append(i_li)
    return arr

def readppm6(name):
# Reads a pgm file (binary/P6)
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]

    f = open(name, "rb")
    header = []
    w=0
    h=0
    while(len(header)<3):
        temp = f.readline()
        if temp[0]==35:
            print("Comment detected in " + name)
        elif len(header)==0:
            assert (temp == b'P6\n')
            header.append(temp.decode("utf-8"))
        elif len(header)==1:
            (w, h) = [int(i.decode("utf-8")) for i in temp.split()]
            header.append([w,h])
        elif len(header)==2:
            header.append([int(temp)])

    data = []
    data.append(header[1])
    data.append(header[2])
    for y in range(h):
        row = []
        for y in range(3*w):
            row.append(ord(f.read(1)))
        data.append(row)
    return data

def readppm(name):
# Reads a ppm file (binary or ascii)
# Returns a 2D array of ints with a header of metada
# Returns [[w,h], [gray], data]
    try:
        try:
            return readppm6(name)
        except:
            return readppm3(name)
    except:
        print("Error reading " + name)
        return
    
def writeppm(file_name, data):
# Writes a ppm file (binary/P6) given ppm data with header

    file_handle = open (file_name, 'wb')
    w = data[0][0]
    h = data[0][1]
    gray = data [1][0]
    pgm_header = f'P6\n{w} {h}\n{gray}\n'
    
    file_handle.write (bytearray (pgm_header, 'ascii')) 

    grayV = np.reshape (data[2:], 3*w*h)

    grayB = struct.pack ('%sB' % len(grayV), *grayV)
    file_handle.write(grayB)
    file_handle.close()
    return


In [37]:
dolls = readppm("images/dolls.ppm")

writeppm("rewrite.ppm", dolls)

dolls_depth = readpgm("images/dolls_disp.pgm")

writepgm("rewritedepth.pgm", dolls_depth)



In [81]:
# Utility Functions

def convertHSV(r, g, b):
# Converts a color from the RGB color model to the HSV color model
# r, g, b values should be in range [0,255]
# Returns a triple of floats in range [0,1] corresponding to (h,s,v)

    rr = r/255
    gg = g/255
    bb = b/255
    maxx = max(rr,gg,bb)
    minn = min(rr,gg,bb)
    delt = maxx-minn
    
    # hue 
    if maxx == rr:
        h = (((gg-bb)/delt) % 6) / 6
    elif maxx == gg:
        h = (((bb-rr)/delt) + 2) / 6
    elif maxx == bb:
        h = (((rr-gg)/delt) + 4) / 6
    else: # delt == 0:
        h = 0
    
    # saturation
    if maxx == 0:
        s = 0
    else:
        s = delt/maxx
        
    # value
    v = maxx
    
    return (h,s,v)

def convertRGB(h, s, v):
# Converts a color from the HSV color model to the RGB color model
# h, s, v values should be in range [0,1]
# Returns a triple of ints in range [0,255] ; (r,g,b)
    H = h*360
    c = v*s
    abss = ((H/60) % 2) - 1
    x = c * (1 - abs(abss))
    m = v - c
    
    if H < 60:
        (rr,gg,bb) = (c,x,0)
    elif H < 120:
        (rr,gg,bb) = (x,c,0)
    elif H < 180:
        (rr,gg,bb) = (0,c,x)
    elif H < 240:
        (rr,gg,bb) = (0,x,c)
    elif H < 300:
        (rr,gg,bb) = (x,0,c)
    else: # H < 360:
        (rr,gg,bb) = (c,0,x)
        
    r = round((rr+m) * 255)
    g = round((gg+m) * 255)
    b = round((bb+m) * 255)
    
    return (r,g,b)

print(convertRGB((100/360),.8,.45))

100.0
(54, 115, 23)


In [ ]:
def bright_points(dmap, p):
# Returns the indices of the brightest values 
    
    return vals

def est_atm_light(dmap, data):
# Given an image and its depth map, finds the atmospheric light
# dmap: a 2D array representing the depth pgm of an image 
# data: a 2D array representing the ppm of an image

    

    A = 0
    
    
    return A